# Collab Filtering - Recommender anwenden 📈

### Jetzt wird Recommendet! 💯

Aber Fangen wir erst mal wieder mit den Imports an:

In [155]:
import pandas as pd
from getpass import getpass
import json
from tqdm import tqdm

### Recommendation Table laden 💾

Als erstes lade ich mir die durch ```Collaborative-Filtering_Matrix-Creator``` erstellte Recommendation Tabelle aus dem gespeicherten CSV wieder rein.

In [156]:
similarity_df = pd.read_csv('item_similarity.csv', index_col = 0)
similarity_df

,spotify:track:000xQL6tZNLJzIrtIgxqSl,spotify:track:00BuKLSAFkaEkaVAgIMbeA,spotify:track:00LfFm08VWeZwB0Zlm24AT,spotify:track:00fNdIFKoMxxt8Hnm2kAKL,spotify:track:00lNx0OcTJrS3MKHcB80HY,spotify:track:00qOE7OjRl0BpYiCiweZB2,spotify:track:00xR9dHhuaNznqB4FSzOlr,spotify:track:015IsLQFXbEm0f541N2qoX,spotify:track:01A7PEPSnmtixFPfB2UTal,spotify:track:01DidSmPasiXdPhDVuaULL,...,spotify:track:7zBPzAjKAqQpcv8F8GCq5s,spotify:track:7zBQRGpYImAdIZc97FNj3V,spotify:track:7zFXmv6vqI4qOt4yGf3jYZ,spotify:track:7zNM46fo01dCBidY4yGNTZ,spotify:track:7zTx8ePYAmPFQuxP3xlXZn,spotify:track:7zVCrzzEJU7u24sbJPXA5W,spotify:track:7zWj09xkFgA9tcV6YhfU6q,spotify:track:7zkLpY72g6lKQbiHDqri1S,spotify:track:7zsw78LtXUD7JfEwH64HK2,spotify:track:7zxRMhXxJMQCeDDg0rKAVo
spotify:track:000xQL6tZNLJzIrtIgxqSl,1.000000,-0.005607,0.021245,-0.003562,0.109532,-0.005130,-0.002907,-0.005046,-0.003562,-0.002907,...,-0.007795,-0.002907,0.027908,-0.004604,0.041288,-0.005607,-0.005046,-0.003904,-0.003562,0.031641
spotify:track:00BuKLSAFkaEkaVAgIMbeA,-0.005607,1.000000,0.033348,0.037933,-0.004736,-0.006820,-0.003865,0.023509,-0.004736,-0.003865,...,-0.010363,-0.003865,-0.007947,-0.006121,-0.005870,-0.007455,0.053727,-0.005190,0.037933,-0.007249
spotify:track:00LfFm08VWeZwB0Zlm24AT,0.021245,0.033348,1.000000,0.027540,-0.006045,-0.008705,-0.004934,0.015222,-0.006045,-0.004934,...,-0.013227,-0.004934,0.050234,-0.007812,0.019653,-0.009516,0.039006,-0.006624,-0.006045,0.012778
spotify:track:00fNdIFKoMxxt8Hnm2kAKL,-0.003562,0.037933,0.027540,1.000000,0.063858,-0.004333,-0.002456,0.090445,-0.003009,-0.002456,...,-0.006584,-0.002456,-0.005049,-0.003889,-0.003729,-0.004736,0.043092,-0.003297,-0.003009,-0.004606
spotify:track:00lNx0OcTJrS3MKHcB80HY,0.109532,-0.004736,-0.006045,0.063858,1.000000,-0.004333,-0.002456,-0.004262,-0.003009,-0.002456,...,0.024325,-0.002456,0.035021,-0.003889,-0.003729,-0.004736,-0.004262,-0.003297,-0.003009,-0.004606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
spotify:track:7zVCrzzEJU7u24sbJPXA5W,-0.005607,-0.007455,-0.009516,-0.004736,-0.004736,-0.006820,-0.003865,-0.006708,-0.004736,-0.003865,...,-0.010363,-0.003865,-0.007947,0.026964,-0.005870,1.000000,0.023509,-0.005190,-0.004736,-0.007249
spotify:track:7zWj09xkFgA9tcV6YhfU6q,-0.005046,0.053727,0.039006,0.043092,-0.004262,-0.006137,-0.003478,0.061033,-0.004262,0.054489,...,-0.009325,-0.003478,-0.007151,0.031209,-0.005282,0.023509,1.000000,-0.004670,-0.004262,0.024539
spotify:track:7zkLpY72g6lKQbiHDqri1S,-0.003904,-0.005190,-0.006624,-0.003297,-0.003297,0.037794,-0.002691,-0.004670,-0.003297,-0.002691,...,-0.007214,-0.002691,-0.005532,-0.004261,-0.004086,-0.005190,-0.004670,1.000000,-0.003297,-0.005047
spotify:track:7zsw78LtXUD7JfEwH64HK2,-0.003562,0.037933,-0.006045,-0.003009,-0.003009,-0.004333,-0.002456,-0.004262,-0.003009,-0.002456,...,-0.006584,-0.002456,-0.005049,-0.003889,-0.003729,-0.004736,-0.004262,-0.003297,1.000000,-0.004606


In [206]:
default_songs = similarity_df.sum().sort_values(ascending=False).to_frame()
# Die ersten len(songs) Vorschläge werden die Lieder selbst dein, daher weg damit!
default_songs.reset_index(level=0, inplace=True)
default_songs.columns = ['TrackURL', 'Rating']
default_songs['Rating'] = default_songs['Rating']/len(default_songs)
default_songs


,TrackURL,Rating
0,spotify:track:6gBFPUFcJLzWGx4lenP6h2,0.017146
1,spotify:track:27GmP9AWRs744SzKcpJsTZ,0.017062
2,spotify:track:5dNfHmqgr128gMY2tc5CeJ,0.016484
3,spotify:track:2qYsSHsYkihWx043HVJQRV,0.016427
4,spotify:track:5CG9Ps5ynNjpKJHmwc95pa,0.016388
...,...,...
5961,spotify:track:1CvoOXxJqC2VHwgjZxjGX5,-0.001472
5962,spotify:track:29CNWt87husOFYXnC8Eb6Y,-0.001644
5963,spotify:track:06wxyCQFJOT0bjvSPMQj7x,-0.001662
5964,spotify:track:5Mw9bXG1dLNhbjofkVS2oR,-0.002047


## Das Recommendation Herz ❤️

```get_similar_songs``` generiert das Rating für Songs zu einem Song 🎵

In [207]:
def get_similar_songs(songURL):
    if songURL not in similarity_df:
        return
    similar_score = similarity_df[songURL]
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score

## Recommendation Herz im Playlist Einsatz 💕

```get_similar_playlist``` generiert mit Hilfe von ```get_similar_songs``` das Rating für Songs zu einer Reihe von Songs, also zu Playlists 🎶



In [250]:
def get_similar_playlist(songs):
    similar_songs = pd.DataFrame()
    if songs == []:
        return default_songs
    if all(song not in default_songs['TrackURL'].values for song in songs):
        return default_songs
    for song in songs:
        similar_songs = similar_songs.append(get_similar_songs(song), ignore_index=True)
    similar_songs = similar_songs.sum().sort_values(ascending=False).to_frame()
    # Die Lieder selbst aus den Vorschläge, weg damit!
    similar_songs = similar_songs.drop(songs, errors='ignore')

    similar_songs.reset_index(level=0, inplace=True)
    similar_songs.columns = ['TrackURL', 'Rating']
    return similar_songs


,TrackURL,Rating
0,spotify:track:5humEOJehpAwMJEJmb0JJ2,0.532143
1,spotify:track:65Q2mv3UlVi9eO70OpsmSe,0.497516
2,spotify:track:6UrKVLhzBYZY6fXmNyCLDi,0.479084
3,spotify:track:5hslUAKq9I9CG2bAulFkHN,0.469839
4,spotify:track:4so0Wek9Ig1p6CRCHuINwW,0.469839
...,...,...
5960,spotify:track:3a1lNhkSLSkpJE4MSHpDu9,-0.010285
5961,spotify:track:7KXjTSCq5nL1LoYtL7XAwS,-0.010796
5962,spotify:track:7yyRTcZmCiyzzJlNzGC9Ol,-0.010796
5963,spotify:track:1xznGGDReH1oQq0xzbwXa3,-0.010901


Dauert ca. 32 minuten

In [251]:

def challenge_set_to_submission(path):
    with open('submission.csv', 'w') as the_file:
        the_file.write('team_info,Sample Submission Team,contactus@email.com\n')
        the_file.write('\n\n')
        with open(path) as f:
            js = f.read()
            challenge_set = json.loads(js)
            
            for playlist in tqdm(challenge_set["playlists"]):

                playlist_track_URIs = []

                for track in playlist["tracks"]:
                    playlist_track_URIs.append(track["track_uri"])

                the_file.write(str(playlist['pid']) + ', ')
                the_file.write(', '.join(get_similar_playlist(playlist_track_URIs)[:500]['TrackURL']))
                the_file.write('\n\n')

challenge_set_to_submission('D:/SpotifyDataset/spotify_million_playlist_dataset_challenge/challenge_set.json')


100%|██████████| 10000/10000 [32:21<00:00,  5.15it/s]


# 🎉 Ja GEIL! 🥳